In [101]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import torch
import torch.nn as nn
import torch.optim as optim

from utils.data_generation import generate_vectors, generate_scalars
from benchmark_functions.sphere import sphere_func
from neural_network_models.feedforward_nn import FeedforwardNN

## Generating data

In [102]:
np.random.seed(1000)
torch.manual_seed(1000)

In [103]:
input_dimention = 5
domain = [-5, 5]

data_set_size = 10_000
training_set_fraction = 0.7
validation_set_fraction = 0.15
test_set_fraction = 0.15

In [104]:
# generate training data
data_set_vectors = generate_vectors(input_dimention, domain, data_set_size)
data_set_scalars = generate_scalars(data_set_vectors, sphere_func)

# separate training data from validation and test data
training_set_vectors, temp_set_vectors, training_set_scalars, temp_set_scalars = train_test_split(
    data_set_vectors, 
    data_set_scalars, 
    test_size = (validation_set_fraction + test_set_fraction), 
    random_state = 42
)

# separate validation data from test data
validation_set_vectors, test_set_vectors, validation_set_scalars, test_set_scalars = train_test_split(
    temp_set_vectors, 
    temp_set_scalars, 
    test_size = (test_set_fraction / (test_set_fraction + validation_set_fraction)), 
    random_state = 42
)

## Data pre-processing

In [105]:
vector_scaler = MinMaxScaler().fit(training_set_vectors)
scalar_scaler = StandardScaler().fit(training_set_scalars.reshape(-1, 1))

In [106]:
training_set_vectors = vector_scaler.transform(training_set_vectors)
validation_set_vectors = vector_scaler.transform(validation_set_vectors)
test_set_vectors = vector_scaler.transform(test_set_vectors)

In [107]:
training_set_scalars = scalar_scaler.transform(training_set_scalars.reshape(-1, 1))
validation_set_scalars = scalar_scaler.transform(validation_set_scalars.reshape(-1, 1))
test_set_scalars = scalar_scaler.transform(test_set_scalars.reshape(-1, 1))

In [108]:
# convert generated data to tensors
training_set_vectors = torch.FloatTensor(training_set_vectors)
training_set_scalars = torch.FloatTensor(training_set_scalars)

validation_set_vectors = torch.FloatTensor(validation_set_vectors)
validation_set_scalars = torch.FloatTensor(validation_set_scalars)

test_set_vectors = torch.FloatTensor(test_set_vectors)
test_set_scalars = torch.FloatTensor(test_set_scalars)

## Creating a model

In [109]:
model = FeedforwardNN(input_neuron_num = input_dimention, h1_neuron_num = 10, output_neuron_num = 1)

loss_func = nn.MSELoss()
loss_optimization_func = optim.Adam(model.parameters(), lr = 0.01)

In [ ]:
learning_epochs = 500

for epoch in range(learning_epochs):
    # pass forward
    training_set_predictions = model(training_set_vectors)
    training_loss = loss_func(training_set_predictions, training_set_scalars)

    # back-propagation
    loss_optimization_func.zero_grad()
    training_loss.backward()
    loss_optimization_func.step()

    # validation step
    model.eval()

    with torch.no_grad():
        validation_set_predictions = model(validation_set_vectors)
        validation_loss = loss_func(validation_set_predictions, validation_set_scalars)
    
    if (epoch + 1) % 100 == 0:
        print(f"[{epoch + 1}/{learning_epochs}] Validation loss: {validation_loss.item():.6f}, Training loss: {training_loss.item():.6f}")

[100/500:5] Validation loss: 1.027023, Training loss: 0.999570
[200/500:5] Validation loss: 1.027304, Training loss: 0.998785
[300/500:5] Validation loss: 1.025284, Training loss: 0.997097
[400/500:5] Validation loss: 1.008454, Training loss: 0.985433
[500/500:5] Validation loss: 0.949289, Training loss: 0.935712


In [111]:
model.eval()

with torch.no_grad():
    test_set_predictions = model(test_set_vectors)
    test_loss = loss_func(test_set_predictions, test_set_scalars)
    print(f"Test loss (MSE): {test_loss.item():.6f}")

Test loss (MSE): 0.966644
